In [90]:
# from textwrap import dedent
from airflow import DAG 
from airflow.operators.python import PythonOperator

from datetime import datetime, timedelta
from finance.stock import (crawl_price,crawl_bargin,crawl_benchmark,crawl_pe)
import time
from finance.process import test_database, date_range
import os
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.models import Variable
from hdfs import InsecureClient
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa
import tempfile
import io

In [114]:
tablename='price'
crawl_func =crawl_price
datetime_object = datetime.strptime('20190107', '%Y%m%d')

dates = date_range(datetime_object, datetime.now())
date = dates[0]

print(f'Crawlar {tablename} {date}')

df = crawl_func(date)


Crawlar price 2019-01-07


In [141]:
    spark = SparkSession.builder.getOrCreate()
    hdfs_path = f'hdfs://namenode:8020/datawarehouse/{tablename}'

In [161]:
new_data_df = df

In [150]:
old_data_df = spark.read.parquet(hdfs_path)
old_data_df

DataFrame[stock_id: string, date: timestamp, 成交股數: bigint, 成交筆數: bigint, 成交金額: bigint, 開盤價: double, 最高價: double, 最低價: double, 收盤價: double, 最後揭示買價: double, 最後揭示賣價: double]

In [153]:
old_data_df_spark = old_data_df_spark.withColumn("date", date_format("date", "yyyy-MM-dd"))

In [ ]:
old_data_df_spark = old_data_df_spark.withColumn("date", date_format("date", "yyyy-MM-dd")).toPandas()
df = pd.concat([old_data_df, new_data_df], ignore_index=True)
df = df.drop_duplicates()
df = df.sort_values(by=['date'])
df = spark.createDataFrame(df)
df.write.parquet(hdfs_path, mode='overwrite')
print(f'Update {tablename} to HDFS')

In [145]:
new_data_df = new_data_df.sort_values(by=['date'])
new_data_df = spark.createDataFrame(new_data_df)
new_data_df.write.parquet(hdfs_path, mode='overwrite')
print(f'Create {tablename} to HDFS')

Create price to HDFS


In [155]:
old_data_df = spark.read.parquet(hdfs_path)
old_data_df_spark = old_data_df_spark.withColumn("date", date_format("date", "yyyy-MM-dd"))
old_data_df = old_data_df_spark.toPandas()


In [160]:
new_data_df

DataFrame[stock_id: string, date: timestamp, 成交股數: bigint, 成交筆數: bigint, 成交金額: bigint, 開盤價: double, 最高價: double, 最低價: double, 收盤價: double, 最後揭示買價: double, 最後揭示賣價: double]

In [162]:
df = pd.concat([old_data_df, new_data_df], ignore_index=True)


In [ ]:
df = df.drop_duplicates()
df = df.sort_values(by=['date'])
df = spark.createDataFrame(df)
df.write.parquet(hdfs_path, mode='overwrite')
print(f'Update {tablename} to HDFS')